# Moving and Rotating a Molecular Cluster:
**Task:** 
1. Move/rotate a Molecule/Atom one by one systematically
2. Move/Rotate randomly

## Water molecule ($H_{2}O$):

Cartesian coordinates [Angstrom]:

    
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000


> **_NOTE:_**   
> `translate` and `rotate` methods return a brand new Molecule object

In [ ]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# importing de Molecule Class
from src.base_molecule import Molecule, Cluster

In [ ]:
# creating a Molecular object. You can use a list, a dictionary 
# (the key MUST be "atoms") or another Molecule object (see notebook: 01)

water_molecule=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]

water = Molecule(water_molecule)

print(water.xyz)

In [ ]:
# Let's create a Cluster adding two more water molecules using 
# the same cartesian coordinates. 

# water3 = Cluster(water_molecule, 2 * water)
water3 = Cluster(water_molecule, water_molecule, water_molecule)

print(water3)
print(water3.xyz)

>### NOTE:
>Creating a `Molecule` object means that its atoms are FIXED and 
>those atoms **can NOT** be MOVED or ROTATED.
>
>On the other hand, creating a `Cluster` object means their individual
>molecules **can** be MOVED or ROTATED

In [ ]:
# let's create a cluster with five (5) water molecules and one atom (Iodine)
iodine =[("I", 0, 0, 0)]

w5_I = Cluster(
    iodine,
    water,
    water,
    water,
    water,
    water,
)

w5_I = Cluster(Molecule(iodine) + 5 * Molecule(water_molecule))

# ALWAYS, you have SIX (6) individual molecules/atoms in your 'w5_I' cluster
print(w5_I)

In [ ]:
# asking for its properties
print(f"\nNumber of individual molecules/atoms: \n{w5_I.total_molecules}")
print(f"\nTotal number of atoms: \n{w5_I.total_atoms}")
print(f"\nAtomic symbols: \n{w5_I.symbols}")
print(f"\nIndividual atomic masses: \n{w5_I.atomic_masses}")
print(f"\nMolecular mass: \n{w5_I.total_mass}")
print(f"\nCoordinates: \n{w5_I.atoms}")
print(f"\nCartesian coordinates: \n{w5_I.coordinates}")

In [ ]:
# let's MOVE 2 units on xy axes the first molecule (I) and rotate 180 deg around z-axis the second molecule (water)
w5_I_moved_rotated = w5_I.translate(0, x=2, y=2, z=0).rotate(1, x=0, y=0, z=180)

print(w5_I_moved_rotated.xyz)

### IMPORTANT FACTS:
- After translating a molecule, a new object is created where the moved 
molecule in the same position. The same is true for any rotation
- Rotation are around the center of mass of the target molecule

In [ ]:
# let's visualize its random process FREEZING Iodine
from copy import deepcopy

# creating a brand new copy when 'I' will NOT be moved or rotated 
w5 = deepcopy(w5_I)

print(w5.sphere_radius)
print(w5.sphere_center)

# setting a sphere radius for the Cluster spherical boundary conditions
w5.sphere_radius = 10

print("\nnew sphere radius")
print(w5.sphere_radius)
print(w5.sphere_center)

!pip install py3Dmol
import py3Dmol

In [ ]:
from numpy import random
from copy import deepcopy

# choosing a fixed seed
random_gen = random.default_rng(1234)

# to save snapshot and show as a movie
w5_xyz = ""

total_steps = 10000

# max step size to move a molecule 
step = w5.sphere_radius

for i in range(total_steps):
    # molecule to be selected between [0, w.total_molecules]
    mol = random_gen.choice(w5.total_molecules)

    # angle between [0, 360)
    ax = random_gen.uniform() * 360
    ay = random_gen.uniform() * 360
    az = random_gen.uniform() * 360

    # moving between [-step, +step]
    tx = step * (random_gen.uniform() - 0.5)
    ty = step * (random_gen.uniform() - 0.5)
    tz = step * (random_gen.uniform() - 0.5)

    # saving coordinates as a string
    # w5_xyz += w5.xyz
    if i % 100 == 0: 
        w5_xyz += w5.xyz

    w5 = w5.translate(mol, x=tx, y=ty, z=tz).rotate(
        mol, x=ax, y=ay, z=az
    )
    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )

# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
print(w5.xyz)

In [ ]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 3Dmol.js viewer in a notebook.
!pip install py3Dmol
import py3Dmol

In [ ]:
# 20% to check visually that everything is inside
sr = w5.sphere_radius * 1.2

# sphere center
cx, cy, cz = w5.sphere_center
sc = {"x": cx, "y": cy, "z": cz}

# axes
x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

# starting visualization
xyz_view = py3Dmol.view(width=700, height=500)#, linked=False, viewergrid=(2,2))
xyz_view.addModelsAsFrames(w5_xyz,'xyz')
# xyz_view.setStyle({'stick': {}})
xyz_view.setStyle({'sphere': {'radius': 0.8}})
xyz_view.addSphere({'center': sc, 
        'radius': sr, 
        'color' :'yellow',
        'alpha': 0.5,
        })

# animation base on several XYZ coordinates snapshots
xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

# cartesian 3D axes
xyz_view.addLine(x_axis)
xyz_view.addLine(y_axis)
xyz_view.addLine(z_axis)

xyz_view.addLabel("x", {
        'position':x_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("y", {
        'position':y_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })
xyz_view.addLabel("z", {
        'position':z_axis["end"],
        'inFront':'true',
        'fontSize':20,
        'showBackground':'false',
        'fontColor': 'black',
        })

xyz_view.zoomTo()
xyz_view.show()
